In [1]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 4.38 MiB | 23.60 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [6]:
import os
os.chdir('/content/gpt-2')
!pip install -r requirements.txt

     |████████████████████████████████| 92kB 7.2MB/s 
     |████████████████████████████████| 604kB 25.1MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=94f202647edd8ccc7230957848b484961a6f4dbbe1ece44b7b673a2aa7d16761
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
  Created wheel for regex: filename=regex-2017.4.5-cp37-cp37m-linux_x86_64.whl size=534361 sha256=6c58287e016912b2aaa47a5d5b1b9f1f1f900f8c59c9aa99842f8a65f0a88f4e
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but

In [7]:
%tensorflow_version 1.x 
import tensorflow as tf

TensorFlow 1.x selected.


In [8]:
!python download_model.py '345M'

Fetching checkpoint: 1.00kit [00:00, 1.01Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 4.05Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.01Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:58, 24.3Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 9.64Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 4.50Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 2.79Mit/s]                                                       


In [10]:
os.chdir('/content/gpt-2/src')

In [11]:
!export PYTHONIOENCODING=UTF-8

In [14]:
import json
import os
import numpy as np
import tensorflow as tf

In [15]:
import model, sample, encoder

In [17]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [19]:
interact_model('345M',None,1,1,300,1,0,'/content/gpt-2/models/')